### Imports and HTML-content

In [1]:
%run helper.py

In [2]:
from IPython.display import display, HTML
point_to_dist_angle = """

"""

In [3]:
from scipy.stats import multivariate_normal

def calc_likelihoods_for_distributions_and_points(matrix_dist, matrix_points):
    means = matrix_dist[:, 0]
    covariances = matrix_dist[:, 1:]
    likelihoods = np.array([multivariate_normal(mean=means[i], cov=covariances[i]).pdf(matrix_points) for i in range(len(means))])
    
    return likelihoods.T 

### Code

### Build Graph

In [4]:
@save_params
def create_graph_data(csv_file_name, iteration):
    classes_dict = {
        'goomba': 0,
        'mario': 1,
        'cloud': 2,
        'ground': 3,
        'bush': 4,
        'box': 5,
        'pipe': 6
    }
    
    class_names, boxes = get_classnames_boxes_from_csv(csv_file_name, iteration)
    num_nodes = len(boxes)
    edge_connections = cartesian_product_for_nodes(range(num_nodes))
    node_features = []
    matrix = np.empty((0, 2))
    normal_dist = [0, 0, 1, 0, 0, 1] # mu1, mu2, sig00, sig01, sig10, sig11
    dataset_number = int(csv_file_name.split('/')[-1].split('.')[0])

    if num_nodes == 1:
        box = boxes[0]
        width, height = abs(box['left'] - box['right']), abs(box['top'] - box['bottom'])
        # node_feature: (normal-distribution, class-label, x-val, y-val, width, height, dataset_number, iteration, id)
        node_features.append((*normal_dist, classes_dict[class_names[0]], box['center_x'], \
                              box['center_y'], width, height, dataset_number, iteration, 0))
        
        node_features = torch.tensor(node_features, dtype=torch.float)
        
        return Data(
            x=node_features,
            edge_index=torch.tensor([0]),
            edge_attr=torch.tensor([0])
        )
        
    for i, box in enumerate(boxes):
        new_row = np.array([[box['center_x'], box['center_y']]])
        matrix = np.vstack((matrix, new_row))

        width, height = abs(box['left'] - box['right']), abs(box['top'] - box['bottom'])
        # node_feature: (normal-distribution, class-label, x-val, y-val, width, height, dataset_number, iteration, id)
        node_features.append((*normal_dist, classes_dict[class_names[i]], box['center_x'], \
                              box['center_y'], width, height, dataset_number, iteration, i))
    
    dists, angles = dist_angle_from_matrix(matrix, edge_connections)
    
    node_features = torch.tensor(node_features, dtype=torch.float)
    edge_connections = torch.tensor(edge_connections)
    edges_features =  torch.tensor(np.stack((dists, angles), axis=-1))
    
    data = Data(
        x=node_features,
        edge_index=edge_connections.t().contiguous(),
        edge_attr=edges_features
    )

    return data

In [5]:
dataset = []
for i in range(4):
    csv_file_name = f"/workspaces/jupyterlite/content/pytroch-geometric/mario-tracking-data/{i:04d}.csv"
    df = pd.read_csv(csv_file_name)
    num_iterations = int(df.iloc[-1]['iteration']) # we start at index 1, so no need for `+ 1`
    
    for iteration in range(1, num_iterations):
        graph_data = create_graph_data(csv_file_name, iteration)
        dataset.append(graph_data)

train_dataset = dataset[:int(0.8 * len(dataset))]
test_dataset = dataset[int(0.8 * len(dataset)):]

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)

/workspaces/jupyterlite/content/pytroch-geometric/helper.py:73: RuntimeWarning: invalid value encountered in divide
  return np.linalg.norm(val_edge_pairs, axis=1), np.rad2deg(np.arctan(val_edge_pairs[:, 1] / val_edge_pairs[:, 0]))
/workspaces/jupyterlite/content/pytroch-geometric/helper.py:73: RuntimeWarning: divide by zero encountered in divide
  return np.linalg.norm(val_edge_pairs, axis=1), np.rad2deg(np.arctan(val_edge_pairs[:, 1] / val_edge_pairs[:, 0]))
/usr/local/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
int(csv_file_name.split('/')[-1].split('.')[0])

3

#### Message Passing

In [7]:
class SimpleGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super().__init__()
        self.conv1 = GATConv(in_channels, out_channels, heads=1, add_self_loops=False)

    def forward(self, x, edge_index, target_node):
        out = self.conv1(x, edge_index)

#### Loss

The Loss will be the SSE of the likelihoods that the points of frame i+1 belong to the distributions that were created from the graph on frame i.


In [8]:
def calc_point_indices_to_distributions(distributions, points, treshhold_likelihood = 0):
    if points.shape[0] == 0:
        return np.zeros(distributions.shape[0]) - 1
    
    if points.shape[0] == 1 and distributions.shape[0] == 1:
        likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, points)
    
        if likelihood_entries < treshhold_likelihood: return np.array([-1])
        return np.array([0])

    likelihoods = calc_likelihoods_for_distributions_and_points(distributions, points)
    sorted_indices = np.argsort(likelihoods, axis=0)
    ranks = np.zeros_like(likelihoods, dtype=int)
    n_rows, n_cols = likelihoods.shape
    ranks[sorted_indices, np.arange(n_cols)] = np.tile(np.arange(n_rows), (n_cols, 1)).T
    
    mask_binary = np.array((n_rows - ranks) <= n_cols, dtype=int)
    cumsum_array = np.cumsum(mask_binary, axis=0)
    
    s = min(n_rows, n_cols)
    extended_likelihood_entries = np.zeros((s+1, n_cols + 1))
    
    mask  = np.array((n_rows - ranks) <= n_cols)
    
    points_to_consider = np.where(np.any(mask, axis=1))[0]
    filtered_points = points[points_to_consider]
    
    likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, filtered_points)
    likelihood_indicies_to_filter = likelihood_entries < treshhold_likelihood
    # punish points that should be filtered out
    likelihood_entries[likelihood_indicies_to_filter] = 1e-50
    
    inv_percentage_of_likelihood = np.log(likelihood_entries)/ np.sum(np.log(likelihood_entries), axis=0)
    # percentage_of_likelihood = (1 / inv_percentage_of_likelihood) / np.sum( (1 / inv_percentage_of_likelihood), axis=0)
    # percentage_of_likelihood = inv_percentage_of_likelihood
    
    cost_matrix = np.log(inv_percentage_of_likelihood)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    max_index = max(col_ind) + 1
    points_to_distributions = np.zeros(max_index, dtype=row_ind.dtype) - 1
    
    points_to_distributions[col_ind] = row_ind

    # if punished points still come up ahead, set the association to -1
    for point_index, distribution_index in enumerate(col_ind):
        if likelihood_indicies_to_filter[point_index, distribution_index]:
            points_to_distributions[point_index] = -1
    
    return points_to_distributions

In [9]:
def classes_points_distributions_ids_from_graph(graph):
    graph = graph.x
    return graph[:, 6], graph[:, :6], graph[:, 7:9], graph[:, -1]

In [10]:
def calculate_optimial_matching_for_distributions_and_points(graph, csv_file_name, iteration, threshhold=0):
        classes_dict = {
        'goomba': 0,
        'mario': 1,
        'cloud': 2,
        'ground': 3,
        'bush': 4,
        'box': 5,
        'pipe': 6
    }
    

In [11]:
def indicies_of_filterted_array_entries(filtered_indicies, relative_indicies):
    return np.where(filtered_indicies)[0][relative_indicies]

In [12]:
def calculate_sse_for_distributions_and_points(graph_a, graph_b, mappign):
    node_attr_a = classes_points_distributions_ids_from_graph(graph_a)
    node_attr_b = classes_points_distributions_ids_from_graph(graph_b)

    # likelihoods = calc_likelihoods_for_distributions_and_points(node_attr_a[1][mappings[0, :], node_attr_b[2][mappings[1, :])
    
    # return 

In [13]:
# @save_params
# def find_mapping_of_two_graphs(graph_a, graph_b):
#     node_attributes_a = classes_points_distributions_ids_from_graph(graph_a)
#     node_attributes_b = classes_points_distributions_ids_from_graph(graph_b)

#     total_mappings = torch.empty((0, 2), dtype=torch.int)

#     for label in classes_points_distributions_ids_from_graph(graph_b)[0].unique():
#         dist_filter = (node_attributes_a[0] == label)
#         dist_indicies = np.copy(dist_filter)
#         points_filter = (node_attributes_b[0] == label)
#         point_indicies = np.copy(points_filter)
#         distributions = node_attributes_a[1][dist_indicies]
#         distributions = np.array(distributions.reshape(distributions.shape[0], -1, 2)) # to have the correct format
#         distributions[:, 0, :] += np.array(node_attributes_a[2][dist_indicies])
#         points = node_attributes_b[2][point_indicies]
#         num_dist_entries = torch.sum(dist_filter)
        
#         mappings = torch.zeros((num_dist_entries, 2)) - 1
#         mappings[:, 0] = torch.tensor((node_attributes_a[-1])[dist_filter], dtype=torch.int)
        
#         dist_to_point_mapping = calc_point_indices_to_distributions(distributions, points)
        
#         for i, dest_index in enumerate(dist_to_point_mapping):
#             if dest_index == -1: continue
#             relative_point_index = indicies_of_filterted_array_entries(points_filter, i)
#             mappings[dest_index, 1] = torch.tensor((node_attributes_b[-1])[relative_point_index], dtype=torch.int)

#         total_mappings = torch.cat((total_mappings, mappings), dim=0)

#     return total_mappings

@save_params
def find_mapping_of_two_graphs(graph_a, graph_b):
    node_attributes_a = classes_points_distributions_ids_from_graph(graph_a)
    node_attributes_b = classes_points_distributions_ids_from_graph(graph_b)

    total_mappings = torch.empty((0, 2), dtype=torch.int)

    for label in classes_points_distributions_ids_from_graph(graph_b)[0].unique():
        dist_filter = (node_attributes_a[0] == label)
        dist_indicies = np.copy(dist_filter)
        points_filter = (node_attributes_b[0] == label)
        point_indicies = np.copy(points_filter)
        distributions = node_attributes_a[1][dist_indicies]
        distributions = np.array(distributions.reshape(distributions.shape[0], -1, 2))  # to have the correct format
        distributions[:, 0, :] += np.array(node_attributes_a[2][dist_indicies])
        points = node_attributes_b[2][point_indicies]
        num_dist_entries = torch.sum(dist_filter)
        
        mappings = torch.full((num_dist_entries, 2), -1, dtype=torch.int)
        mappings[:, 0] = torch.tensor((node_attributes_a[-1])[dist_filter], dtype=torch.int)
        
        dist_to_point_mapping = calc_point_indices_to_distributions(distributions, points)
        
        for i, dest_index in enumerate(dist_to_point_mapping):
            if dest_index == -1:
                continue
            relative_point_index = indicies_of_filterted_array_entries(points_filter, i)
            mappings[dest_index, 1] = torch.tensor((node_attributes_b[-1])[relative_point_index], dtype=torch.int)
        
        total_mappings = torch.cat((total_mappings, mappings), dim=0)

    return total_mappings

In [14]:
graph_a = train_dataset[700]
graph_b = train_dataset[700]

In [15]:
find_mapping_of_two_graphs(graph_a, graph_b)

/tmp/ipykernel_629158/1774780086.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mappings[:, 0] = torch.tensor((node_attributes_a[-1])[dist_filter], dtype=torch.int)


NameError: name 'points' is not defined

In [ ]:
graph_a.x

In [ ]:
distributions

In [ ]:
mappigs = find_mapping_of_two_graphs(graph_a, graph_b)
mappings

In [ ]:
graph_b.x

In [ ]:
[mappings[:, 0]]

In [ ]:
classes_dict = {
    'goomba': 0,
    'mario': 1,
    'cloud': 2,
    'ground': 3,
    'bush': 4,
    'box': 5,
    'pipe': 6
}

csv_data = get_classnames_boxes_from_csv(csv_file_name, 50)
class_ids = np.array([classes_dict[class_name] for class_name in csv_data[0]])
boxes_coordinates = np.empty((0, 2))

for box_values in csv_data[1]:
    box_coordinates =  np.array([box_values['center_x'], box_values['center_y']])
    boxes_coordinates = np.vstack((boxes_coordinates, box_coordinates))

class_ids, boxes_coordinates

In [ ]:
node_attributes_a = classes_points_distributions_ids_from_graph(graph_a)
node_attributes_b = classes_points_distributions_ids_from_graph(graph_b)
node_attributes_a

In [ ]:
def likelihood_of_distributions_from_center_of_distributions_to_points(distributions, center_distributions, points):
    normalized_points = points - center_distribution
    return calc_point_indices_to_distributions(distributions, normalized_points)

In [ ]:
labels = torch.tensor([1, 2, 1, 1, 3])
matrix = np.array([
    [1, 1],
    [2, 3],
    [4, 1],
    [1, 4],
    [5, 3]
])

matrix[labels == 1]

In [ ]:
classes_points_distributions_ids_from_graph(graph_b)[0].unique()
labels.unique()

In [ ]:
matching_nodes = torch.empty((0, 2))

In [ ]:
# for label in node_attributes_a[0].unique():
label = 1
dist_filter = (node_attributes_a[0] == label)
dist_indicies = np.copy(dist_filter)
points_filter = (node_attributes_b[0] == label)
point_indicies = np.copy(points_filter)
distributions = node_attributes_a[1][dist_indicies]
distributions = np.array(distributions.reshape(distributions.shape[0], -1, 2)) # to have the correct format
distributions[:, 0, :] += np.array(node_attributes_a[2][point_indicies])
points = node_attributes_b[2][point_indicies]
num_dist_entries = torch.sum(dist_filter)

mappings = torch.zeros((num_dist_entries, 2)) - 1
mappings[:, 0] = (node_attributes_a[-1])[dist_filter]

dist_to_point_mapping = calc_point_indices_to_distributions(distributions, points)

for i, dest_index in enumerate(dist_to_point_mapping):
    if dest_index == -1: continue
    relative_point_index = indicies_of_filterted_array_entries(points_filter, i)
    mappings[dest_index, 1] = (node_attributes_b[-1])[relative_point_index]

In [ ]:
mappings

In [ ]:
dist_filter

In [ ]:
torch.sum(dist_filter)

In [ ]:
dist_to_point_mapping

In [ ]:
mappings

In [ ]:
dist_to_point_mapping

In [ ]:
dist_indicies.shape[0]

In [ ]:
indicies_of_filterted_array_entries(points_filter, calc_point_indices_to_distributions(distributions, points))

In [ ]:
matching_nodes = torch.zeros((distributions.shape[0], 2)) - 1
matching_nodes

In [ ]:
distributions = np.array([
    [[3, 0], [1, 0], [0, 1]],
    [[2, 0], [1, 0.5], [0.5, 1]]
])

points = np.random.uniform(-10, 10, (2, 2))

# distributions[:, 0, :] += points
# distributions

# calc_likelihoods_for_distributions_and_points(distributions, points)
calc_point_indices_to_distributions(distributions, points)

In [ ]:
calc_point_indices_to_distributions(distributions, points)

In [ ]:
node_attributes_b[0] == label

In [ ]:
np.where(node_attributes_b[0] == label)[0]

In [ ]:
# classes_dict = {
#     'goomba': 0,
#     'mario': 1,
#     'cloud': 2,
#     'ground': 3,
#     'bush': 4,
#     'box': 5,
#     'pipe': 6
# }

# class_names, boxes = get_classnames_boxes_from_csv(csv_file_name, iteration)
# num_nodes = len(boxes)
# edge_connections = cartesian_product_for_nodes(range(num_nodes))
# node_features = []
# matrix = np.empty((0, 2))
# normal_dist = [0, 0, 1, 0, 1, 0] # mu1, mu2, sig00, sig01, sig10, sig11

# if num_nodes == 1:
#     box = boxes[0]
#     width, height = abs(box['left'] - box['right']), abs(box['top'] - box['bottom'])
#     node_features.append((*normal_dist, classes_dict[class_names[0]], width, height))
    
# for i, box in enumerate(boxes):
#     new_row = np.array([[box['center_x'], box['center_y']]])
#     matrix = np.vstack((matrix, new_row))

#     width, height = abs(box['left'] - box['right']), abs(box['top'] - box['bottom'])
#     node_features.append((*normal_dist, classes_dict[class_names[i]], width, height))

# dists, angles = dist_angle_from_matrix(matrix, edge_connections)

# node_features = torch.tensor(node_features, dtype=torch.float)
# edge_connections = torch.tensor(edge_connections)
# edges_features =  torch.tensor(np.stack((dists, angles), axis=-1))